In [1]:
import transformers

In [3]:
long_text = """
Advancements in Renewable Energy Technologies
The last decade has seen remarkable advancements in renewable energy technologies, signaling a significant shift towards more sustainable and environmentally friendly energy sources. These advancements have not only contributed to the reduction of greenhouse gas emissions but also enhanced the efficiency and cost-effectiveness of renewable energy production.

Solar Power: Solar energy technology has made significant leaps, especially in photovoltaic (PV) cell efficiency and the development of solar thermal power. The introduction of perovskite solar cells offers a cheaper and potentially more efficient alternative to the traditional silicon-based cells. Innovations such as floating solar farms are maximizing the use of water bodies for energy production without occupying valuable land space.

Wind Energy: Wind energy has also seen considerable improvements, with the development of larger, more efficient turbine designs. Offshore wind farms are becoming increasingly common, benefiting from stronger and more consistent wind speeds available at sea. The use of digital twins technology in wind turbines helps in optimizing their operation and maintenance, significantly increasing energy output.

Hydroelectric Power: Advancements in hydroelectric technology have focused on minimizing environmental impacts and enhancing efficiency. The development of fish-friendly turbines has reduced the ecological footprint of hydroelectric dams. Small-scale hydroelectric projects are becoming more popular, offering a viable energy solution for remote communities without the extensive environmental disruption caused by larger dams.

Geothermal Energy: Geothermal energy has expanded its reach beyond traditional hotspots thanks to enhanced geothermal systems (EGS), which allow for the exploitation of geothermal resources in a wider range of geographical locations. This technology involves injecting water into hot rocks deep underground to produce steam, which can then be used to generate electricity.

Bioenergy: The bioenergy sector has made strides in producing biofuels from non-food sources, reducing competition with food production. The development of advanced biofuels from algae and cellulosic materials presents a promising future for sustainable fuel production. Biogas technology has also advanced, improving the efficiency of converting organic waste into energy.

Storage and Integration: One of the critical challenges with renewable energy is its intermittent nature. Recent advancements in energy storage technologies, such as lithium-ion batteries, flow batteries, and hydrogen storage, are addressing this issue, enabling more reliable and continuous energy supply. Moreover, the integration of renewable energy sources into the grid has been facilitated by smarter grid management systems, which balance supply and demand more effectively.

Policy and Market Development: Alongside technological advancements, policy and market developments have played a crucial role in the adoption of renewable energy. Incentives like tax credits, feed-in tariffs, and renewable energy certificates have made investments in renewable technologies more attractive. The international commitment to reducing carbon emissions, as seen in agreements like the Paris Agreement, continues to drive innovation and adoption of renewable energy technologies.

In conclusion, the advancements in renewable energy technologies over the past decade are paving the way for a more sustainable and environmentally friendly future. While challenges remain, particularly in terms of storage and grid integration, the continued innovation and supportive policy landscape present a positive outlook for the widespread adoption of renewable energy.
"""

long_text = long_text.replace("\n", "")

In [2]:
from transformers import pipeline

In [4]:
extractive_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", min_length=75, max_length=200)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
extractive_summary = extractive_summarizer(long_text)

In [8]:
extractive_summary[0]['summary_text']

'The last decade has seen remarkable advancements in renewable energy technologies. Solar energy technology has made significant leaps, especially in photovoltaic (PV) cell efficiency and the development of solar thermal power. Offshore wind farms are becoming increasingly common, benefiting from stronger and more consistent wind speeds available at sea. Policy and market developments have played a crucial role in the adoption of renewable energy.'

Evaluating with ROUGE

In [9]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=7631ca4e0ad62fb4fed532edd2a0b37ed6b252b16c3107fd4949f08b41eaae7b
  Stored in directory: /Users/ronaldmatovu/Library/Caches/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [10]:
import evaluate

In [11]:
rouge_evaluator = evaluate.load('rouge')

In [12]:
#Evaluate exact match strings
reference_text=["This is the same string"]
predict_text=["This is the same string"]

In [13]:
eval_results=rouge_evaluator.compute(predictions=predict_text, 
                                     references=reference_text)

In [14]:
f"Results for exact match strings: {eval_results}"

"Results for exact match strings: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}"

In [15]:
#Evaluate no-match strings
reference_text=["This is the different string"]
predict_text=["Google can predict warm weather"]

eval_results=rouge_evaluator.compute(predictions=predict_text, 
                                     references=reference_text)

In [16]:
f"Results for no-match strings: {eval_results}"

"Results for no-match strings: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}"

In [17]:
# Evaluate summary
eval_results=rouge_evaluator.compute(
    predictions=[extractive_summary[0].get("summary_text")], 
    references=[long_text])

In [18]:
f"Results for the generated summary: {eval_results}"

"Results for the generated summary: {'rouge1': 0.22107081174438684, 'rouge2': 0.20450606585788558, 'rougeL': 0.21761658031088082, 'rougeLsum': 0.21761658031088082}"